This notebook will house the updated table 'Opportunity Youth Status by Age' of the 2016 report 'Opportunity Youth in the Road Map Project Region' with current data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import psycopg2
import pandas as pd

In [4]:
from src.data import sql_utils

In [5]:
DBNAME = "opportunity_youth"

In [6]:
conn = psycopg2.connect(dbname=DBNAME)

I want to see the opportunity youth count per PUMA within South King County

In [7]:
oy_by_puma = pd.read_sql("""SELECT SUM(pwgtp) AS oy_count, n.puma_name
               FROM pums_2017 p
               JOIN puma_names_2010 n
               ON p.puma = n.puma
               WHERE p.puma BETWEEN '11610' AND '11615'
               AND (p.esr = '3' OR p.esr = '6')
               AND p.sch = '1'
               AND p.agep BETWEEN '16' AND '24'
               GROUP BY n.puma_name
               
               ;""", conn)
oy_by_puma

,oy_count,puma_name
0,1853.0,"King County (Central)--Renton City, Fairwood, ..."
1,1977.0,"King County (Far Southwest)--Federal Way, Des ..."
2,1210.0,"King County (Southeast)--Maple Valley, Covingt..."
3,2006.0,King County (Southwest Central)--Kent City ...
4,1530.0,King County (Southwest)--Auburn City & Lakelan...
5,2038.0,"King County (West Central)--Burien, SeaTac, Tu..."


## We have decided to include the PUMAs for South King County as reported by

In [8]:
total_oy = oy_by_puma.sum()
total_oy

oy_count                                                 10614
puma_name    King County (Central)--Renton City, Fairwood, ...
dtype: object

## Querying pums_2017 table with the PUMA codes classified as in South King County to return the population of opportunity youth in the age group from 16-18.

## The total Opportunity Youth population for the age group of 16-18.

In [9]:
first_pop_by_puma = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name
                FROM pums_2017 p
                JOIN puma_names_2010 n
                ON p.puma = n.puma
                WHERE (agep BETWEEN '16' AND '18')
                AND (p.puma BETWEEN '11610' AND '11615')
                AND (p.sch = '1')
                AND (p.esr != '1') 
                AND (p.esr != '4')
                GROUP BY n.puma_name
                ;""", conn)

oy_pop_to_8teen = first_pop_by_puma['pop_count'].sum()
oy_pop_to_8teen

1815.0

## The total Opportunity Youth population for the age group of 19-21.

In [10]:
second_pop_by_puma = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name
                FROM pums_2017 p
                JOIN puma_names_2010 n
                ON p.puma = n.puma
                WHERE agep BETWEEN '19' AND '21'
                AND p.puma BETWEEN '11610' AND '11615'
                AND (p.sch = '1')
                AND (p.esr != '1' AND p.esr != '4')
                GROUP BY n.puma_name
                ;""", conn)
oy_pop_to_2one = second_pop_by_puma['pop_count'].sum()
oy_pop_to_2one

4083.0

## The total Opportunity Youth population for the age group of 22-24.

In [11]:
third_pop_by_puma = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name
                FROM pums_2017 p
                JOIN puma_names_2010 n
                ON p.puma = n.puma
                WHERE agep BETWEEN '22' AND '24'
                AND p.puma BETWEEN '11610' AND '11615'
                AND (p.esr != '1' AND p.esr != '4')
                AND (p.sch = '1')
                GROUP BY n.puma_name
                ;""", conn)
oy_pop_to_2four = third_pop_by_puma['pop_count'].sum()
oy_pop_to_2four

5217.0

## The total Opportunity Youth population for all age groups 16-24.

In [12]:
oy_pop_total = (oy_pop_to_8teen + oy_pop_to_2one + oy_pop_to_2four)
oy_pop_total

11115.0

In [13]:
first_total_pop = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name
                FROM pums_2017 p
                JOIN puma_names_2010 n
                ON p.puma = n.puma
                WHERE agep BETWEEN '16' AND '18'
                AND p.puma BETWEEN '11610' AND '11615'
                GROUP BY n.puma_name
                ;""", conn)
total_pop_to_8teen = first_total_pop['pop_count'].sum()
total_pop_to_8teen

30141.0

In [14]:
second_total_pop = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name
                FROM pums_2017 p
                JOIN puma_names_2010 n
                ON p.puma = n.puma
                WHERE agep BETWEEN '19' AND '21'
                AND p.puma BETWEEN '11610' AND '11615'
                GROUP BY n.puma_name
                ;""", conn)
total_pop_to_2one = second_total_pop['pop_count'].sum()
total_pop_to_2one

25486.0

In [15]:
third_total_pop = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name
                FROM pums_2017 p
                JOIN puma_names_2010 n
                ON p.puma = n.puma
                WHERE agep BETWEEN '22' AND '24'
                AND p.puma BETWEEN '11610' AND '11615'
                GROUP BY n.puma_name
                ;""", conn)
total_pop_to_2four = third_total_pop['pop_count'].sum()
total_pop_to_2four

30256.0

## The total population for all age groups 16-24 as defined in the Road Map Project region.

In [16]:
population_total = (total_pop_to_8teen + total_pop_to_2one + total_pop_to_2four)
population_total

85883.0

## Here, I chose to utilize the variables esr(employment status), cow(class of worker), and schl(educational attainment).  I am querying these to see the amount of opportunity youth between the ages of 16-18 that are currently working without a diploma.  ESR equalling 1(civilian employed, at work) or 2(armed forces, at work) COW not equalling 9(unemployed)

## The total population for the age group 16-18 that are working without a diploma.

In [24]:
work_no_diploma1 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE p.puma BETWEEN '11610' AND '11615'
                                            AND (p.esr = '1' OR p.esr = '4')
                                            AND (p.cow != '9')
                                            AND NOT (p.schl > '15') 
                                            AND p.agep BETWEEN '16' AND '18'
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
work_no_diploma1
work_no_dip_18_total = work_no_diploma1['pop_count'].sum()
work_no_dip_18_total

4292.0

## The total population for the age group 19-21 that are working without a diploma.

In [25]:
work_no_diploma2 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE p.puma BETWEEN '11610' AND '11615'
                                            AND (p.esr = '1' OR p.esr = '4')
                                            AND (p.cow != '9')
                                            AND NOT (p.schl > '15') 
                                            AND p.agep BETWEEN '19' AND '21'
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
work_no_dip_21 = work_no_diploma2['pop_count'].sum()
work_no_dip_21

1588.0

## The total population for the age group 22-24 that are working without a diploma.

In [26]:
work_no_diploma3 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE p.puma BETWEEN '11610' AND '11615'
                                            AND (p.esr = '1' OR p.esr = '4')
                                            AND (p.cow != '9')
                                            AND NOT (p.schl > '15') 
                                            AND p.agep BETWEEN '22' AND '24'
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn) 
work_no_dip_24 = work_no_diploma3['pop_count'].sum()
work_no_dip_24

1663.0

## The total population for all age groups 16-24 that are currently working without a dipoloma.

In [80]:
work_no_dip_total = (work_no_dip_18_total + work_no_dip_21 + work_no_dip_24)
work_no_dip_total

7543.0

# Querying the entire population within the Roadmap Project region that are not classified as Opportunity Youth by removing the ESR, COW, and SCHL variables.

## The total population for the age group 16-18 that are not considered Opportunity Youth.

In [29]:
not_oy_youth1 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name
                FROM pums_2017 p
                JOIN puma_names_2010 n
                ON p.puma = n.puma
                WHERE agep BETWEEN '16' AND '18'
                AND (p.puma BETWEEN '11610' AND '11615')
                GROUP BY n.puma_name
                ;""", conn)
not_oy_to_8teen = (total_pop_to_8teen - work_no_dip_18_total - oy_pop_to_8teen)
not_oy_to_8teen

24034.0

## The total population for the age group 19-21 that are not considered Opportunity Youth.

In [30]:
not_oy_youth2 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name
                FROM pums_2017 p
                JOIN puma_names_2010 n
                ON p.puma = n.puma
                WHERE agep BETWEEN '19' AND '21'
                AND (p.puma BETWEEN '11610' AND '11615')
                GROUP BY n.puma_name
                ;""", conn)
not_oy_to_2one = (total_pop_to_2one - work_no_dip_21 - oy_pop_to_2one)
not_oy_to_2one

19815.0

## The total population for the age group 22-24 that are not considered Opportunity Youth.

In [33]:
not_oy_youth3 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name
                FROM pums_2017 p
                JOIN puma_names_2010 n
                ON p.puma = n.puma
                WHERE agep BETWEEN '22' AND '24'
                AND (p.puma BETWEEN '11610' AND '11615')
                GROUP BY n.puma_name
                ;""", conn)
not_oy_to_2four = (total_pop_to_2four - work_no_dip_24 - oy_pop_to_2four)
not_oy_to_2four

23376.0

## The total population for all age groups 16-24 that are not classified as Opportunity Youth.

In [32]:
not_oy_youth_total = (not_oy_to_8teen + not_oy_to_2one + not_oy_to_2four)
not_oy_youth_total

67225.0

## Querying to find the total population for the age group 16-18 that are classified as Opportunity Youth and do not have a diploma.

In [34]:
oy_no_dip1 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name
                FROM pums_2017 p
                JOIN puma_names_2010 n
                ON p.puma = n.puma
                WHERE (agep BETWEEN '16' AND '18')
                AND (p.puma BETWEEN '11610' AND '11615')
                AND (p.sch = '1')
                AND (p.schl NOT BETWEEN '16' AND '23')
                AND (p.esr != '1') 
                AND (p.esr != '4')
                GROUP BY n.puma_name
                ;""", conn)

oy_no_dip_8teen = oy_no_dip1['pop_count'].sum()
oy_no_dip_8teen

916.0

## The total population for the age group 19-21 that are classified as Opportunity Youth and do not have a diploma.

In [35]:
oy_no_dip2 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE (agep BETWEEN '19' AND '21')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl NOT BETWEEN '16' AND '23')
                                            AND (p.esr != '1') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
oy_no_dip_2one = oy_no_dip2['pop_count'].sum()
oy_no_dip_2one

1135.0

## The total population for the age group 22-24 that are classified as Opportunity Youth and do not have a diploma.

In [116]:
oy_no_dip3 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE (agep BETWEEN '22' AND '24')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl NOT BETWEEN '16' AND '23')
                                            AND (p.esr != '1') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
oy_no_dip_2four = oy_no_dip3['pop_count'].sum()
oy_no_dip_2four 

1391.0

## The total population for all age groups 16-24 that are classified as Opportunity Youth and do not have a diploma.

In [132]:
oy_no_dip_total = (oy_no_dip_8teen + oy_no_dip_2one + oy_no_dip_2four)
oy_no_dip_total

3442.0

## The total population for the age group 16-18 that are classified as Opportunity Youth having a diploma or GED.

In [36]:
oy_dip_ged1 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE (agep BETWEEN '16' AND '18')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl = '16' OR p.schl = '17')
                                            AND (p.esr != '1') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
oy_dip_ged_8teen = oy_dip_ged1['pop_count'].sum()
oy_dip_ged_8teen 

781.0

## The total population for the age group 19-21 that are classified as Opportunity Youth having a diploma or GED.

In [37]:
oy_dip_ged2 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE (agep BETWEEN '19' AND '21')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl = '16' OR p.schl = '17')
                                            AND (p.esr != '1') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
oy_dip_ged_2one = oy_dip_ged2['pop_count'].sum()
oy_dip_ged_2one 

2183.0

## The total population for the age group 22-24 that are classified as Opportunity Youth having a diploma or GED.

In [38]:
oy_dip_ged3 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE (agep BETWEEN '22' AND '24')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl = '16' OR p.schl = '17')
                                            AND (p.esr != '1') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
oy_dip_ged_2four = oy_dip_ged3['pop_count'].sum()
oy_dip_ged_2four 

2325.0

## The total population for all age groups 16-24 that are classified as Opportunity Youth having a diploma or GED.

In [39]:
oy_dip_ged_total = (oy_dip_ged_8teen + oy_dip_ged_2one + oy_dip_ged_2four)
oy_dip_ged_total

5289.0

## Population for the age group 16-18 that have attended some college but do not possess a degree.

In [40]:
some_col1 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE (agep BETWEEN '16' AND '18')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl = '18' OR p.schl = '19')
                                            AND (p.esr != '1' AND p.esr != '4') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
some_col_8teen = some_col1['pop_count'].sum()
some_col_8teen

118.0

In [44]:
some_col12_query = sql_utils.open_sql_script('13_some_college.sql')
some_col12 = pd.read_sql(some_col12_query, conn)
some_col12_total = some_col12['pop_count'].sum()
some_col12_total

118.0

## Population for the age group 19-21 that have attended some college but do not possess a degree.

In [41]:
some_col2 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE (agep BETWEEN '19' AND '21')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl = '18' OR p.schl = '19')
                                            AND (p.esr != '1') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
some_col_2one = some_col2['pop_count'].sum()
some_col_2one

649.0

## Population for the age group 22-24 that have attended some college but do not possess a degree.

In [126]:
some_col3 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE (agep BETWEEN '22' AND '24')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl = '18' OR p.schl = '19')
                                            AND (p.esr != '1') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
some_col_2four = some_col3['pop_count'].sum()
some_col_2four

1074.0

In [103]:
some_col_total = (some_col_8teen + some_col_2one + some_col_2four)
some_col_total

20201.0

In [127]:
ass_deg1 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE (agep BETWEEN '16' AND '18')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl >= '20')
                                            AND (p.esr != '1') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
ass_deg8teen = ass_deg1['pop_count'].sum()
ass_deg8teen

0

In [128]:
ass_deg2 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                           WHERE (agep BETWEEN '19' AND '21')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl >= '20')
                                            AND (p.esr != '1') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
ass_deg2one = ass_deg2['pop_count'].sum()
ass_deg2one

116.0

In [129]:
ass_deg3 = pd.read_sql("""SELECT SUM(pwgtp) AS pop_count, n.puma_name, p.puma
                                            FROM pums_2017 p
                                            JOIN puma_names_2010 n
                                            ON p.puma = n.puma
                                            WHERE (agep BETWEEN '22' AND '24')
                                            AND (p.puma BETWEEN '11610' AND '11615')
                                            AND (p.sch = '1')
                                            AND (p.schl >= '20')
                                            AND (p.esr != '1') 
                                            AND (p.esr != '4')
                                            GROUP BY n.puma_name, p.puma
                                            ;""", conn)
ass_deg2four = ass_deg3['pop_count'].sum()
ass_deg2four

427.0

In [130]:
ass_degree_total = (ass_deg8teen + ass_deg2one + ass_deg2four)
ass_degree_total

543.0